### Default Setting 

In [13]:
from llama_index import Document, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index.storage.storage_context import StorageContext
# from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
from llama_index import Document, VectorStoreIndex 
from llama_index.node_parser import SentenceSplitter
import chromadb
import pandas as pd 
import openai
import os
import getpass
import glob 

In [743]:
default_path = os.getcwd()
model_path = os.path.join(default_path, '../../models')
model_dir = os.path.join(model_path, "mistral_origin")
data_path = os.path.join(default_path, '../../data')
rulebook_path = os.path.join(data_path, 'pdf', 'rules')

In [605]:
file_list = glob.glob(f'{rulebook_path}/*.pdf')
file_list[5]

'/rag/jupyter/embedding/../../data/pdf/rules/복지제도_가이드.pdf'

In [94]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [95]:
chroma_client = chromadb.HttpClient(host="192.168.0.146")

In [96]:
chroma_client.list_collections()

[Collection(name=deposit),
 Collection(name=data),
 Collection(name=card),
 Collection(name=loan)]

In [97]:
data_collection = chroma_client.get_or_create_collection("data")

In [98]:
data_collection

Collection(name=data)

In [432]:
data_store = ChromaVectorStore(chroma_collection=data_collection)
data_storage = StorageContext.from_defaults(vector_store=data_store)

In [433]:
page_no = 0

In [1168]:
documents = SimpleDirectoryReader(input_files=[file_list[5]]).load_data()

In [1169]:
documents[page_no]

Document(id_='4da8a95e-96a5-4af7-845b-5f327d9f9c94', embedding=None, metadata={'page_label': '1', 'file_name': '복지제도_가이드.pdf', 'file_path': '/rag/jupyter/embedding/../../data/pdf/rules/복지제도_가이드.pdf', 'file_type': 'application/pdf', 'file_size': 372327, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-06', 'last_accessed_date': '2024-03-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='c22b35b0daa4b0977b196ac6d276b3959c0308680361e6c6f329fa0d6702d4a8', text='   \n페이지０  \n \n \n \n \n \n \n \n \n \n \n   \n[ 순     서 ] \n \n1. 동호회  지원 \n2. 신규입사자  / 직급자  모임 \n3. 회식비  지원 \n4. 직원소개  장려금  제도 \n5. 경조금  및 경조화환  지원 \n6. 장례서비스  지원 \n7. 사내강사비  지원 \n8. 선택적  복지포인트  제도  \n9. 휴양시설  지원 \n10. 가정의  날 \n11. 직무발명  제도 \n12. 학기준비금  / 학자금  지원 \n13. 리프레시  휴가 지원 \

In [1170]:
file_list[2]

'/rag/jupyter/embedding/../../data/pdf/rules/경조금지급규정.pdf'

In [1171]:
if documents[0].text[0].isdigit() and documents[1] != '.':
    print(f'페이지번호: {documents[0].text[0]}쪽')

In [1172]:
import re

def text_cleanse(text):
    text = re.sub('[^A-Za-z0-9\'\"\-가-힣(){}\\n[]]', '', text)
    for num in num_mapper.keys():
        text = text.replace(num, num_mapper[num])
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

In [1173]:
num_mapper = dict({'①':'제1항', '②':'제2항', '③':'제3항', '④':'제4항', '⑤':'제5항', '⑥':'제6항',\
                   '⑦':'제7항', '⑧': '제8항', '⑨': '제9항', '⑩': '제10항', '⑪': '제11항', '⑫': '제12항',\
                  '⑬':'제13항', '⑭':'제14항', '⑮':'제15항', '⑯':'제16항', '⑰':'제17항', '⑱':'제18항', '⑲':'제19항', '⑳':'제20항'})

In [1174]:
for idx in range(len(documents)):
    documents[idx].text = text_cleanse(documents[idx].text)

#### Document 재정의 

In [1187]:
text = documents[idx].text 
documents[0].metadata['page_label']

'1'

In [1190]:
txt_tmp = documents[0].copy()
txt_tmp.text.splitlines(True)

['페이지０ \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 ' \n',
 '[ 순 서 \n',
 ' \n',
 '1. 동호회 지원 \n',
 '2. 신규입사자 / 직급자 모임 \n',
 '3. 회식비 지원 \n',
 '4. 직원소개 장려금 제도 \n',
 '5. 경조금 및 경조화환 지원 \n',
 '6. 장례서비스 지원 \n',
 '7. 사내강사비 지원 \n',
 '8. 선택적 복지포인트 제도 \n',
 '9. 휴양시설 지원 \n',
 '10. 가정의 날 \n',
 '11. 직무발명 제도 \n',
 '12. 학기준비금 / 학자금 지원 \n',
 '13. 리프레시 휴가 지원 \n',
 ' \n',
 ' \n',
 '복지제도 가이드']

In [1191]:
def get_text(document):
    first_line = ''
    txt_lines = document.text.splitlines(True)
    
    if document.text.startswith(document.metadata['page_label']):
        # print('페이지 번호를 포함한 문서입니다.')
        document.text = document.text.lstrip(document.metadata['page_label'])
    
    elif document.text.startswith('페이지'):
        print(f"페이지 번호: {document.metadata['page_label']}")
        document.text = document.text.lstrip(f"페이지:  {int(document.metadata['page_label']) - 1}")
    document.text = re.sub(r'\r\n{2,} \r\n{2, }', '\n\n', document.text)
    return document.text

In [1192]:
import re 

text = '제 1 장 제1장 제 1장 제 2 장 제2장 제 2장' 
# re.findall(r'제.*[0-9].*장', text)
re.findall(r'제.+2.+장', text)

['제 1 장 제1장 제 1장 제 2 장 제2장 제 2장']

In [1193]:
text = '제 1 장 제1장 제 1장 제 2 장 제2장 제 2장 제 4장' 
re.findall(r'제 *1 *장', text)

['제 1 장', '제1장', '제 1장']

In [1194]:
def get_start_point(documents):
    s_point = 1
    for doc in documents:
        if len(re.findall(r'제 *1 *장', doc.text)) != 0 and (len(re.findall(r'목 *차', doc.text)) == 0 and len(re.findall(r'차 *례', doc.text)) == 0): 
            '''
            print(re.findall(r'제 *1 *장', doc.text))
            print(re.findall(r'목 *차', doc.text))
            print(re.findall(r'차 *례', doc.text))'''
            s_point = doc.metadata['page_label']
            break 
    return int(s_point) - 1

In [1195]:
documents[0].metadata['page_label']

'1'

In [1196]:
s_point = get_start_point(documents)
s_point

0

In [1197]:
doc = get_text(documents[1])
print(doc)

１ 
 
 
1. 동호회 지원 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
2. 신규입사자 / 
직급자 모임 
혜택 건전한 취미 생활로 단결을 도모할 수 있는 단체 지원 (5인 이상) 
대상 당사 임직원 
지원금액 월 30,000원 (1인당) 
세부내용 제1항 운영동호회 
 명칭 활동사항 회비 비고 
F.G.C 골프 동호회 
30,000 원/월 참석인원당지급 
동호회별 신청 마라탕 마라톤 , 등산 
공채모임 공채 기수 모임 
제2항 운영보고 : 아마란스 를 통해 모임의 일자/장소/목적/결과를 공지하여야 함 
(아마란스 에 공지하는 것으로 결과 및 운영보고를 갈음함 ) 
 - 모임의 개최는 최소 1주일 전에 아지트를 통해 공지 
 - 모임 당일의 행사 내용 및 결과에 대해서는 익일까지 아지트에 공지 
 - 참석인원 전원의 사진을 아지트에 올려주셔야 함. 
신청방법 제1항 전자결재 신청 
- 이카운트 >> 전자결재 >> 지출결의서 (기타경비 ) 
- 결재라인 : (1차) 기안자 >> 담당자 (김태영 과장) 
 (2차) 김용화 수석 / (참조) 경영전략실 
지급일 매월 15일 지급 
담당자 김태영 과장 / 02)799 -2580 / tykim@finger.co.kr 
 
혜택 신규입사 및 직급자별 친목&단결 도모 
대상 당사 임직원 
지원금액 - 
세부내용 제1항 신규입사자 모임 
- 분기 마지막 월 마지막 주 
제2항 직급자 모임 
- 짝수 달 마지막 주 
신청방법 아마란스 공지사항에 별도 공지 
지급일 - 
담당자 김태영 과장 / 02)799 -2580 / tykim@finger.co.kr 
 복지제도 TITLE


In [1082]:
documents[0].metadata

{'page_label': '1',
 'file_name': '복지제도_가이드.pdf',
 'file_path': '/rag/jupyter/embedding/../../data/pdf/rules/복지제도_가이드.pdf',
 'file_type': 'application/pdf',
 'file_size': 372327,
 'creation_date': '2024-03-06',
 'last_modified_date': '2024-03-06',
 'last_accessed_date': '2024-03-21'}

In [971]:
def get_doc_content(document):  
    '''
    input: document (페이지 단위로 분할된 document object) 
    output: document_list (장 단위로 분할된 document objects) 
    '''
    doc_list = []; meta_info = dict(); 
    print(re.findall(r'제 *[0-9] *장', document.text))

In [972]:
doc_list = []

for idx, document in enumerate(documents):
    id_name = document.metadata['file_name'].split('.')[0]
    print(int(document.metadata['page_label']) - 1)
    if int(document.metadata['page_label']) - 1 >= s_point:
        '''
        doc = Document(text=document.text,
                       doc_id=f"{id_name}_doc_{idx}",
                       metadata={"page_label": int(document.metadata['page_label']) - 1, "file_name": document.metadata['file_name']},
                       excluded_llm_metadata_keys = ['page_label', 'file_name']
                )
        doc_list.append(doc)'''
        get_doc_content(document)

0
[]
1
[]


In [952]:
doc_list[0].text

IndexError: list index out of range

In [953]:
print(doc_list[1].text)

IndexError: list index out of range

In [841]:
from llama_index.node_parser import SentenceSplitter 

In [842]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [843]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name, embed_batch_size=32)

In [844]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [845]:
index = VectorStoreIndex.from_documents(
    doc_list, service_context=service_context, storage_context=data_storage
)

In [846]:
data_collection.count()

188

In [669]:
from llama_index.retrievers import VectorIndexRetriever 
from llama_index.vector_stores import SimpleVectorStore
from llama_index.query_engine import RetrieverQueryEngine 
from llama_index.postprocessor import SimilarityPostprocessor 
from llama_index.postprocessor import SentenceTransformerRerank 
from llama_index.postprocessor import KeywordNodePostprocessor 
from llama_index.postprocessor import SimilarityPostprocessor, CohereRerank
from llama_index.schema import Node, NodeWithScore 

In [671]:
retriever = VectorIndexRetriever(
    index = index,
    service_context=service_context,
    similarity_top_k = 10, 
    verbose=True
)

In [754]:
def get_retrieved_nodes(
    retriever, query_str, vector_top_k=10, similarity_cutoff=0.6, reranker_top_n=3, service_context=None, with_reranker=False
):
    query_bundle = QueryBundle(query_str)   # query bundle 생성 
    retrieved_nodes = retriever.retrieve(query_bundle)   # 유사도가 제일 높은 node 추출 
    node_postprocessors = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)   # 전처리  - 유사 점수 기준 Cutoff 
    processed_nodes = node_postprocessors.postprocess_nodes(retrieved_nodes)
    
    if with_reranker:   # 재순위화 
        reranker = SentenceTransformerRerank(
            model='bongsoo/albert-small-kor-cross-encoder-v1',
            top_n=reranker_top_n,
        )
        reranked_nodes = reranker.postprocess_nodes(
            processed_nodes, query_bundle
        )
    return reranked_nodes

In [715]:
with_reranker = True
cutoff = 0.2

query = '교통비의 정의 기준은 ?' # queries[0]

In [716]:
import time
from llama_index import QueryBundle

start = time.time()
retrieved_nodes = retriever.retrieve(query)
nodes = get_retrieved_nodes(retriever, query, similarity_cutoff=cutoff, service_context=service_context, with_reranker=with_reranker)
end = time.time() - start 
print(end)

1.19443941116333


In [717]:
nodes[0].node.relationships['1'].metadata

{'page_label': 1, 'file_name': '신여비교통비.pdf'}

In [718]:
for key, value in nodes[0].node.relationships.items():
    print(f"key: {key}, value: {value}", end='\n\n')

key: 1, value: node_id='신여비교통비_doc_1' node_type=<ObjectType.DOCUMENT: '4'> metadata={'page_label': 1, 'file_name': '신여비교통비.pdf'} hash='c0395b03779d2b24a5f5c1385ee48dccef329514cfc4293667cf5c8785f1c405'

key: 2, value: node_id='6ab7a5cd-8915-41aa-9ec6-3be743ac8f9e' node_type=<ObjectType.TEXT: '1'> metadata={'page_label': 1, 'file_name': '신여비교통비.pdf'} hash='ca8f23d11a07e5b410cff08e815078185f044ca1d079c82e79c95a9d89f5679f'

key: 3, value: node_id='987143ad-4a77-40cb-a10d-34c7b3715313' node_type=<ObjectType.TEXT: '1'> metadata={} hash='3a63d0dee2494b16d5bf5c31cb32c6e756a5eb06ea396675f703f977fd96b90b'



In [719]:
def get_info(retrieved_node):
    '''
    id, text, score 반환 
    '''
    id = retrieved_node.node.relationships['1'].node_id
    name = retrieved_node.node.relationships['1'].metadata['name']
    txt = retrieved_node.node.text 
    score = retrieved_node.score 
    return [id, name, txt, score] 

In [720]:
print(len(nodes))

3


In [721]:
contexts = [node.text for node in nodes]
scores = [node.score for node in nodes]
contexts, scores 

(['제2항 상위 직책을 대리하거나, 동행하는 경우에는 그 직급의 출장 경비를 지급할 수 있다.\n제7조 (지급의 제한)\n제1항 회사소유 또는 임차한 시설이 있거나 회사가 교통편을 제공하는 경우 이를 이용함을 원칙으로\n한다.\n제2항 장기체재에 따른 숙소 제공 시에는 숙박비를 지원하지 아니한다.\n제3항 장기출장 시에는 야근 식대 및 교통비를 지원하지 아니한다 .',
  '제18조 (장기출장 시 귀향여비 )\n제1항 해외 장기출장 시 국내 귀향에 따른 왕복항공료를 회사에서 분기당 1회씩 지원한다.\n제2항 국내 장기출장 시 귀향에 따른 교통비를 별표 제1호에 의하여 회사에서 월 2회씩 지원한다 .\n단, 거주지를 출장지로 이전한 경우에는 월 1회로 한다.\n부 칙\n제1조 (시행일)\n제1항 이 규정은 2017년 1월 1일부터 제정, 시행한다 .\n제2항 이 규정 시행일 이전의 위 내용과 관련된 규정은 이 규정의 시행 즉시 효력을 상실한다 .\n부 칙\n제1조 (시행일)\n제1항 이 규정은 2018년 8월 1일부터 개정, 시행한다 .',
  '제1장 총 칙\n제1조 (목적)\n이 규정은 회사의 임.직원 또는 계약에 의해 회사의 업무를 수행하는 자가 회사의 출장명령에 의해\n국.내외를 출장할 때 그 여비의 지급기준에 관한 사항을 정함을 목적으로 한다.\n제2조 (용어정의 )\n이 규정에서 사용하는 용어의 정의는 아래 각 호와 같다.\n1. “교통비”라 함은 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다.\n2. “숙박비”라 함은 1박 이상의 출장 시 숙박에 소요되는 비용을 말한다.\n3. “일비”라 함은 출장 중 발생하는 현지교통비 또는 도시권내 교통비 및 이에 따른 부대비용과\n출장중의 통신비 및 제 잡비 등의 모든 경비를 말한다.\n4. “식비”라 함은 출장 중 출장자의 식사 및 음료 비용 등을 말한다.\n5. “업무상 외출”이라 함은 숙박을 요하지 않는 당일 출장을 말한다.\n6. “장기출장 ”이라 함은 1개월을 초과하여 수행하

In [724]:
print(nodes[2].text)

제1장 총 칙
제1조 (목적)
이 규정은 회사의 임.직원 또는 계약에 의해 회사의 업무를 수행하는 자가 회사의 출장명령에 의해
국.내외를 출장할 때 그 여비의 지급기준에 관한 사항을 정함을 목적으로 한다.
제2조 (용어정의 )
이 규정에서 사용하는 용어의 정의는 아래 각 호와 같다.
1. “교통비”라 함은 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다.
2. “숙박비”라 함은 1박 이상의 출장 시 숙박에 소요되는 비용을 말한다.
3. “일비”라 함은 출장 중 발생하는 현지교통비 또는 도시권내 교통비 및 이에 따른 부대비용과
출장중의 통신비 및 제 잡비 등의 모든 경비를 말한다.
4. “식비”라 함은 출장 중 출장자의 식사 및 음료 비용 등을 말한다.
5. “업무상 외출”이라 함은 숙박을 요하지 않는 당일 출장을 말한다.
6. “장기출장 ”이라 함은 1개월을 초과하여 수행하는 출장을 말한다.
7. “장기체재 ”라 함은 장기출장의 일종으로서 동일장소에서 3개월을 초과하여 수행하는 출장을
말한다.


In [727]:
nodes[0].metadata

{'page_label': 1, 'file_name': '신여비교통비.pdf'}

In [730]:
def get_info(retrieved_node):
    '''
    id, text, score 반환 
    '''
    id = retrieved_node.node.relationships['1'].node_id
    name = retrieved_node.node.relationships['1'].metadata['file_name']
    txt = retrieved_node.node.text 
    score = retrieved_node.score 
    return [id, name, txt, score] 

In [733]:
print(get_info(nodes[0])[2])

제2항 상위 직책을 대리하거나, 동행하는 경우에는 그 직급의 출장 경비를 지급할 수 있다.
제7조 (지급의 제한)
제1항 회사소유 또는 임차한 시설이 있거나 회사가 교통편을 제공하는 경우 이를 이용함을 원칙으로
한다.
제2항 장기체재에 따른 숙소 제공 시에는 숙박비를 지원하지 아니한다.
제3항 장기출장 시에는 야근 식대 및 교통비를 지원하지 아니한다 .


In [753]:
prompt_template = (
    f"""<s>[INST] 질문: {query} \n
    관련 정보: {nodes[2].text} \n 
    관련 정보를 바탕으로 질문에 자연스럽게 답해줘 [/INST] \n"""
)

print(prompt_template)

<s>[INST] 질문: 교통비의 정의 기준은 ? 

    관련 정보: 제1장 총 칙
제1조 (목적)
이 규정은 회사의 임.직원 또는 계약에 의해 회사의 업무를 수행하는 자가 회사의 출장명령에 의해
국.내외를 출장할 때 그 여비의 지급기준에 관한 사항을 정함을 목적으로 한다.
제2조 (용어정의 )
이 규정에서 사용하는 용어의 정의는 아래 각 호와 같다.
1. “교통비”라 함은 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다.
2. “숙박비”라 함은 1박 이상의 출장 시 숙박에 소요되는 비용을 말한다.
3. “일비”라 함은 출장 중 발생하는 현지교통비 또는 도시권내 교통비 및 이에 따른 부대비용과
출장중의 통신비 및 제 잡비 등의 모든 경비를 말한다.
4. “식비”라 함은 출장 중 출장자의 식사 및 음료 비용 등을 말한다.
5. “업무상 외출”이라 함은 숙박을 요하지 않는 당일 출장을 말한다.
6. “장기출장 ”이라 함은 1개월을 초과하여 수행하는 출장을 말한다.
7. “장기체재 ”라 함은 장기출장의 일종으로서 동일장소에서 3개월을 초과하여 수행하는 출장을
말한다. 
 
    관련 정보를 바탕으로 질문에 자연스럽게 답해줘 [/INST] 



In [737]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

In [738]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [744]:
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaTokenizerFast, BitsAndBytesConfig

tokenizer = LlamaTokenizerFast.from_pretrained(os.path.join(model_dir, 'tokenizer'))   # LlamaTokenizer (x)  -> LlamaTokenizerFast (o)
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True) # , device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [748]:
g_device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(g_device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [749]:
generation_config = GenerationConfig(
    temperature=0.8,
    do_sample=True,
    top_p=0.95,
    max_new_tokens=512,
)

In [750]:
gened = model.generate(
    **tokenizer(
        prompt_template,
        return_tensors='pt',
        return_token_type_ids=False
    ).to('cuda'),
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    # streamer=streamer,
)

In [751]:
result_str = tokenizer.decode(gened[0])

start_tag = f"[/INST]"
start_index = result_str.find(start_tag)

if start_index != -1:
    result_str = result_str[start_index + len(start_tag):].strip()

In [752]:
print(result_str)

질문: 교통비의 정의 기준은? 

답변: 
교통비는 경유지와 종착 지점간의 철도, 항공, 선박, 자동차 등의 운임을 말한다. 따라서, 출장 중 이동하는 경로와 수단에 따라 교통비가 지급되는 경우이다.</s>
